# 14. 머신러닝 모델을 제품으로 만들어보자 : MLOps 기초[프로젝트]

## 14-1. 프로젝트: 이제까지 만들었던 모델을 튜닝하고 배포하기!

이번 프로젝트는 여러분들이 배운 프로젝트중에서 하나를 뽑아서 하이퍼파라미터 튜닝을 하고 배포하는 미션입니다!

### 1. 하이퍼파라미터 튜닝

이제까지 진행했던 프로젝트 하나를 뽑아서(예시 : Seq2Seq, GAN, CIFAR10 딥러닝 모델, VGG 16모델) KerasTuner로 하이퍼파라미터 튜닝을 진행합니다. 튜닝한 하이퍼 파라미터로 학습을 진행하고 모델을 SavedModel 형식으로 저장해주세요.

(주의사항)
하이퍼 파라미터를 어느정도 범위로 주는지에 따라서 모델 튜닝속도가 달라집니다.
학습시간을 고려해서 하이퍼 파라미터 튜닝을 세팅해주세요!

In [15]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from tensorflow.keras.datasets import cifar10

import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.utils import to_categorical



In [6]:
# 문제 3-1: cifar10 데이터셋을 로드하고, 훈련 데이터셋에서 20%를 검증 데이터셋으로 분리합니다.
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()
#20%를 검증 데이터셋으로 분리
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)

# 훈련, 검증, 테스트 데이터와 레이블 종류가 몇개인지 출력합니다.
print("전체 학습 데이터: {} 레이블: {}".format(x_train_full.shape, y_train_full.shape))
print("학습 데이터: {} 레이블: {}".format(x_train.shape, y_train.shape))
print("검증 데이터: {} 레이블: {}".format(x_val.shape, y_val.shape))
print("테스트 데이터: {} 레이블: {}".format(x_test.shape, y_test.shape))

전체 학습 데이터: (50000, 32, 32, 3) 레이블: (50000, 1)
학습 데이터: (40000, 32, 32, 3) 레이블: (40000, 1)
검증 데이터: (10000, 32, 32, 3) 레이블: (10000, 1)
테스트 데이터: (10000, 32, 32, 3) 레이블: (10000, 1)


In [7]:
# Data Normalization
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

In [8]:
y_train = to_categorical(y_train, num_classes=10)
y_val = to_categorical(y_val, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [16]:
print("학습 데이터: {} 레이블: {}".format(x_train.shape, y_train.shape))

학습 데이터: (40000, 32, 32, 3) 레이블: (40000, 10)


In [9]:
# cifar10의 분류에 해당하는 'airplane', 'automobile', 'bird', 'cat', 'deer',
# 'dog', 'frog', 'horse', 'ship', 'truck'를 class_name으로 정의합니다.
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [10]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}


In [11]:
def improved_build_model(hp):
    model = tf.keras.Sequential()
    
    # Input Layer
    model.add(tf.keras.Input(shape=x_train.shape[1:], name='inputs'))
    
    # Convolutional Layers with MaxPooling, Batch Normalization, and Dropout
    for i in range(hp.Int('num_layers', min_value=1, max_value=3)):  # Reduce max layers to 3
        model.add(tf.keras.layers.Conv2D(hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
                                         (3,3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(0.5))
    
    # Flatten
    model.add(tf.keras.layers.Flatten())
    
    # Dense Layers with Batch Normalization and Dropout
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes_{i}', values=[32, 64, 128, 256]), activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(0.5))
    
    # Output Layer
    model.add(tf.keras.layers.Dense(10, activation='softmax', name='outputs'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model



In [12]:
# Instantiate the tuner with the improved model building function
improved_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=improved_build_model,
    overwrite=True,
    project_name='improved_keras_tuner'
)

# Train the model
improved_keras_tuner.search(x_train, y_train, validation_data=(x_val, y_val), epochs=3)

Trial 10 Complete [00h 00m 30s]
accuracy: 0.3273000121116638

Best accuracy So Far: 0.6150000095367432
Total elapsed time: 00h 07m 37s


In [13]:
# Get the best model and its summary
best_hps_improved = improved_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
improved_model = improved_build_model(best_hps_improved)
improved_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 30, 30, 128)       3584      
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 15, 15, 128)       0         
 ng2D)                                                           
                                                                 
 batch_normalization_44 (Ba  (None, 15, 15, 128)       512       
 tchNormalization)                                               
                                                                 
 dropout_44 (Dropout)        (None, 15, 15, 128)       0         
                                                                 
 conv2d_26 (Conv2D)          (None, 13, 13, 32)        36896     
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 6, 6, 32)        

In [14]:

save_path = os.getenv('HOME') + '/aiffel5_quest/exploration/aiffel/mlops/node14/model/1'
fname = os.path.join(save_path)
model.save(fname)

NameError: name 'model' is not defined

### 2. 모델 배포하기

하이퍼파라미터 튜닝을 끝낸 모델을 Docker 혹은 WSL2 환경에서 TFServing을 진행해 모델을 배포하세요! 모델 배포가 성공했다면 해당 코드쉘을 캡쳐해서 이미지를 주피터 노트북에 넣어주세요!



```bash
$ echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -
$ sudo apt update
$ sudo apt install tensorflow-model-server
```

### 3. TFLite 모델 만들기

여러분들이 만든 모델을 TFLite 모델로 만들어서 저장하고 서명을 확인하는 메소드까지 입력해주세요!

In [ ]:
load_path = os.getenv('HOME') + '/aiffel5_quest/exploration/aiffel/mlops/node14/model/1'
best_model = tf.keras.models.load_model(load_path)

In [ ]:
best_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_65 (Conv2D)          (None, 30, 30, 102)       2856      
                                                                 
 conv2d_66 (Conv2D)          (None, 28, 28, 42)        38598     
                                                                 
 conv2d_67 (Conv2D)          (None, 26, 26, 67)        25393     
                                                                 
 conv2d_68 (Conv2D)          (None, 24, 24, 37)        22348     
                                                                 
 conv2d_69 (Conv2D)          (None, 22, 22, 52)        17368     
                                                                 
 flatten_11 (Flatten)        (None, 25168)             0         
                                                                 
 dense_16 (Dense)            (None, 256)             

### 4.(추가미션) 박찬성님의 TFX 프로젝트 분석해보기

![](https://d3s0tskafalll9.cloudfront.net/media/images/TFX.max-800x600.png)

[박찬성님의 TFX 프로젝트]

박찬성님의 프로젝트 : semantic-segmentation-ml-pipeline
https://github.com/deep-diver/semantic-segmentation-ml-pipeline

이번 노드에서 TensorFlow Extended에 대한 컴포넌트 소개를 했지만 각각 컴포넌트가 어떻게 작동하는지 코드를 보지 않았습니다.
현재 Machine Learning GDE(Google Developers Expert)이자 Hugging Face Fellow로 활동하고 계신 박찬성님은 전세계적으로 손꼽히는 TFX를 잘 다루는 유저입니다.
위에 올린 깃헙주소의 경우 박찬성님께서 진행한 TFX 프로젝트중 하나로 Semantic Segmentation 모델을 사용한 ML 파이프라인 제작 프로젝트입니다.

해당 프로젝트를 분석한다면 여러분들께서 TFX에 대한 이해도가 높아져서 TFX의 매력에 빠지게 될거라는 생각이 듭니다!

만일 1번부터 3번까지 프로젝트를 다 진행하셨다면 추가미션을 통해 TFX를 이해하는 시간을 가졌으면 좋겠습니다!

오늘도 수고하셨습니다!